In [1]:
import re
import time
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
from fuzzymatcher import link_table, fuzzy_left_join

In [9]:

import fresh_data.get_datasets
import importlib
importlib.reload(fresh_data.get_datasets)
from fresh_data.get_datasets import *

In [10]:
polarization = load_polarization_data()
polarization.head()

,representative,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,last_means,bioguide_id,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,nokken_poole_dim1,nokken_poole_dim2,state_name
34784,"CALLAHAN, Herbert Leon (Sonny)",106,House,15090,41,1,AL,200,1.0,C000052,...,2021.0,0.373,0.202,-156.97696,0.830,842.0,61.0,0.290,0.263,Alabama
34785,"CRAMER, Robert E. (Bud), Jr.",106,House,29100,41,5,AL,100,1.0,C000868,...,NaN,-0.132,0.612,-241.95350,0.755,861.0,102.0,-0.124,0.540,Alabama
34786,"EVERETT, Robert Terry",106,House,29300,41,2,AL,200,1.0,E000268,...,NaN,0.414,0.528,-120.12012,0.866,834.0,43.0,0.447,0.563,Alabama
34787,"BACHUS, Spencer T., III",106,House,29301,41,6,AL,200,1.0,B000013,...,NaN,0.387,0.228,-197.21883,0.796,863.0,83.0,0.305,0.103,Alabama
34788,"HILLIARD, Earl Frederick",106,House,29302,41,7,AL,100,1.0,H000621,...,NaN,-0.555,0.569,-217.22676,0.773,845.0,83.0,-0.575,0.651,Alabama


In [11]:
polarization.congress.max()

116

In [12]:
polarization.state_name.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [7]:
polarization[polarization["congress"]==106]

,representative,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,last_means,bioguide_id,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,nokken_poole_dim1,nokken_poole_dim2,state_name
35225,"CALLAHAN, Herbert Leon (Sonny)",107,House,15090,41,1,AL,200,1.0,C000052,...,2021.0,0.373,0.202,-60.44528,0.90300,594.0,22.0,0.354,0.132,Alabama
35226,"CRAMER, Robert E. (Bud), Jr.",107,House,29100,41,5,AL,100,1.0,C000868,...,NaN,-0.132,0.612,-176.67485,0.76500,660.0,71.0,-0.083,0.464,Alabama
35227,"EVERETT, Robert Terry",107,House,29300,41,2,AL,200,1.0,E000268,...,NaN,0.414,0.528,-79.89400,0.88300,645.0,33.0,0.467,0.530,Alabama
35228,"BACHUS, Spencer T., III",107,House,29301,41,6,AL,200,1.0,B000013,...,NaN,0.387,0.228,-75.17126,0.88900,640.0,24.0,0.368,0.137,Alabama
35229,"HILLIARD, Earl Frederick",107,House,29302,41,7,AL,100,1.0,H000621,...,NaN,-0.555,0.569,-214.38777,0.72200,658.0,88.0,-0.524,0.645,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35664,"RYAN, Paul D.",107,House,29939,25,1,WI,200,1.0,R000570,...,NaN,0.556,-0.250,-122.57333,0.83074,661.0,56.0,0.499,-0.299,Wisconsin
35665,"BALDWIN, Tammy",107,House,29940,25,2,WI,100,1.0,B001230,...,NaN,-0.492,-0.134,-157.37660,0.78898,664.0,78.0,-0.570,-0.172,Wisconsin
35666,"GREEN, Mark",107,House,29941,25,8,WI,200,1.0,G000545,...,NaN,0.456,-0.294,-123.49722,0.83400,679.0,58.0,0.502,-0.329,Wisconsin
35667,"BARRETT, Thomas M.",107,House,39315,25,5,WI,100,1.0,B000177,...,NaN,-0.380,-0.507,-138.67879,0.80700,647.0,60.0,-0.426,-0.426,Wisconsin
